## ആവശ്യമായ ലൈബ്രററികൾ ഇംപോർട്ട് ചെയ്യുക
Phi-4 മോഡൽ ലോഡ് ചെയ്യാനും ഉപയോഗിക്കാനുമുള്ള PyTorch மற்றும் transformers ലൈബ്രററികൾ ഇംപോർട്ട് ചെയ്യുക.


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

## റാൻഡം സീഡ് സജ്ജമാക്കുക
വ്യത്യസ്ത ഓട്ടങ്ങളിൽ പുനരാവർത്തിക്കാവുന്ന ഫലങ്ങൾ ഉറപ്പാക്കാൻ റാൻഡം സീഡ് സജ്ജമാക്കുക.


In [ ]:
torch.random.manual_seed(0)

## Phi-4-mini-flash-reasoning മോഡൽ మరియు ടോക്കനൈസർ ലോഡ് ചെയ്യുക
Hugging Face-ൽ നിന്നുള്ള Microsoft Phi-4-mini-flash-reasoningmodel-ഉം അതിന്റെ അനുയോജ്യമായ ടോക്കനൈസറും ലോഡ് ചെയ്യുക. മോഡൽ വേഗമായ ഇൻഫറൻസിനായി CUDA-യിൽ ലോഡ് ചെയ്യപ്പെടും.


In [ ]:
model_id = "microsoft/Phi-4-mini-flash-reasoning"
model = AutoModelForCausalLM.from_pretrained(
   model_id,
   device_map="cuda",
   torch_dtype="auto",
   trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

## ഇൻപുട്ട് സന്ദേശം തയ്യാറാക്കുക
രണ്ടാം ഡിഗ്രി സമവാക്യത്തോട് ബന്ധപ്പെട്ട ഒരു ഗണിത പ്രശ്നം ഉൾപ്പെടുത്തിയുള്ള ഒരു സംഭാഷണ സന്ദേശം സൃഷ്ടിച്ച് അത് മോഡലിന്റെ ചാറ്റ് ടെംപ്ലേറ്റ് ഉപയോഗിച്ച് ഫോർമാറ്റ് ചെയ്യുക.


In [ ]:
messages = [{
   "role": "user",
   "content": "How to solve 3*x^2+4*x+5=1?"
}]   
inputs = tokenizer.apply_chat_template(
   messages,
   add_generation_prompt=True,
   return_dict=True,
   return_tensors="pt",
)

## പ്രതികരണം സൃഷ്ടിക്കുക
temperature, top_p പോലുള്ള നിർദ്ദിഷ്ട പാരാമീറ്ററുകൾ ഉപയോഗിച്ച് ഔട്ട്പുട്ടിലെ നിയന്ത്രിത യാദൃശ്ചികതയ്ക്കായി മോഡലിൽ നിന്നുള്ള ഒരു പ്രതികരണം സൃഷ്ടിക്കുക.


In [ ]:
outputs = model.generate(
   **inputs.to(model.device),
   max_new_tokens=32768,
   temperature=0.6,
   top_p=0.95,
   do_sample=True,
)

## ഔട്ട്പുട്ട് ടെക്സ്റ്റിലേക്ക് ഡീകോഡ് ചെയ്യുക

സൃഷ്ടിച്ച ടോക്കൺ ശൃംഖലകൾ മനുഷ്യന് വായിക്കാവുന്ന ടെക്സ്റ്റിലേക്കായി തിരിച്ച് മാറ്റുക; മോഡലിന്റെ മറുപടിയാണ് മാത്രം കാണിക്കാൻ വേണ്ടതിനാൽ ആ പരമ്പരയിൽ ഉണ്ടായിരുന്ന മൂല ഇൻപുട്ട് ടോക്കണുകൾ ഒഴിവാക്കുക.


In [ ]:
outputs = tokenizer.batch_decode(outputs[:, inputs["input_ids"].shape[-1]:])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
ഡിസ്‌ക്ലെയിമർ:
ഈ രേഖ AI പരിഭാഷാ സേവനമായ [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് പരിഭാഷപ്പെടുത്തിയതാണ്. ഞങ്ങൾ കൃത്യതയെ ലക്ഷ്യമിടുന്നുണ്ടെങ്കിലും, സ്വയംപ്രക്രിയയിലൂടെ നടപ്പിലാക്കപ്പെടുന്ന പരിഭാഷകളിൽ പിശകുകൾ അല്ലെങ്കിൽ അസത്യതകൾ ഉണ്ടാകാമെന്ന点 ദയവായി ശ്രദ്ധിക്കുക. മാതൃഭാഷയിൽ ഉള്ള മൗലിക രേഖയാണ് പ്രാമാണിക ഉറവിടമായി കണക്കാക്കേണ്ടത്. നിർണായകമായ വിവരങ്ങൾക്ക് പ്രൊഫഷണൽ മനുഷ്യപരിഭാഷ ശുപാർശ ചെയ്യപ്പെടുന്നു. ഈ പരിഭാഷ ഉപയോഗപ്പെടുത്തുന്നതാൽ ഉണ്ടാകാവുന്ന ഏതെങ്കിലും തെറ്റിദ്ധാരണങ്ങൾക്കോ തെറ്റുവ്യാഖ്യാനങ്ങളിലോ ഞങ്ങൾക്ക് ഉത്തരവാദിത്വം ഉണ്ടാകില്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
